# Step 1, DataJoin: To clean and merge 311 service request data with the NYC Street Tree Census
** Note that throughout this and other jupyter notebooks, the following terminology is used:
- falls: refer to fallen branches (from the 311 service request dataset) since 2015
- service requests: refer to 311 service requests for damaged trees or overhanging branches (but NOT fallen branches) since 2015
- tree census: refers to the 2015 NYC Street Tree Census

Packages:

In [4]:
# Import packages
import pandas as pd
import numpy as np
import geopandas
import math

Functions

In [10]:
''' Function to match a 311 service request tree with a 2015 Street Tree Census tree_id
        input is the address, latitude, and longitude of the 311 service request and the Street Tree Census dataframe
        output is the tree_id '''

# create the dataframe that is called in the functions default
tree_census_df=pd.DataFrame()

# the function
def assign_tree_id(address,lat,lon,df=tree_census_df.copy()):
    # Find all street trees assigned to the same address as the 311 service request
    tempdf = df[df.address==address]
    # If multiple street trees are assigned to the address, find the one closest to the service request by latitude-longitude distance and output its tree_id
    if tempdf.count()['tree_id']>=2:
        dis=np.sqrt(np.square(tempdf.latitude-lat)+np.square(tempdf.longitude-lon))
        tree_index = np.argmin(np.array(dis))
        return tempdf.tree_id.iloc[tree_index]
    # If no street trees have the address output nan
    elif tempdf.count()['tree_id']==0:
        return np.nan
    # If only one street tree has the address, output that tree_id
    else:
        return tempdf.tree_id.to_numpy()[0]

### First, import and organize the 2015 NYC Street Tree Census data.
The tree census data is stored in a GIS shapefile, so it needs to be imported as a geopandas geodataframe and converted to a pandas dataframe.

Many of the columns can be dropped, like political boundaries (e.g. state assembly district), information about the data entry, and tree specifics (e.g. species latin name).

In [6]:
# Import tree census data as a geopandas geodataframe
tree_census_geodf = geopandas.read_file('Data/Trees/2015 Street Tree Census - Tree Data/geo_export_f9cac9c3-66c4-45b6-899b-823a270d8754.shp')

# Convert tree census data into a pandas dataframe, remove un-needed columns, and remove geometry columns
tree_census_df = pd.DataFrame(tree_census_geodf.drop(columns='geometry'), copy=True)
tree_census_df.drop(['boro_ct', 'borocode', 'boroname', 'brnch_ligh',
       'brnch_othe', 'brnch_shoe', 'cb_num', 'cncldist','nta',
       'nta_name','spc_latin', 'st_assem', 'st_senate', 'state','stump_diam','user_type', 'x_sp', 'y_sp', 'zip_city'],axis=1,inplace=True)

tree_census_df.dropna(inplace=True)

### Next, import 311 Tree fall service requests and assign each a street tree census tree_id
Several unneeded location columns can be dropped from the 311 requests.

Only falls after the tree census date (2015) will be used.  Only branch falls, not entire tree falls, will be used.  All requests for trees in parks (not street trees) will be removed.

In [11]:
# Import 311 tree fall service request data and drop unneeded columns
fall_df = pd.read_csv('Data/311/Tree_Fall_311.csv',quoting=3)
fall_df.drop([
       'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Status', 'Borough',
       'X Coordinate (State Plane)', 'Y Coordinate (State Plane)'],axis=1, inplace=True)

# Remove all falls before 2015 (before the street tree census)
fall_df['Created Date'] = pd.to_datetime(fall_df['Created Date'])
fall_df = fall_df.where(fall_df['Created Date'] > '2015-01-01')

# Remove all falls that occured in parks, and not in street trees
fall_df = fall_df.where(fall_df['Location Type'] != 'Park')

# Only keep branch falls (eliminate whole tree falls)
fall_df=fall_df[fall_df.Descriptor=='Branch or Limb Has Fallen Down']

KeyboardInterrupt: 

Match each 311 branch fall service request with a street tree census tree_id using the assign_tree_id function defined above.  This matches trees based on address and lat-lon distance.

In [ ]:
# In order to match service request tree falls to street tree census tree_ids, remove service requests without addresses or coordinates
fall_df = fall_df[fall_df.Incident Address.isna()==False]
fall_df = fall_df[fall_df.Latitude.isna()==False]
fall_df = fall_df[fall_df.Longitude.isna()==False]

In [ ]:
# Find closest tree from the street tree census and then assign its tree_index to the 311 data frame
fall_tree_ids_list = [assign_tree_id(address,lat,lon) for address,lat,lon in zip(fall_df.Incident Address,fall_df.Latitude,fall_df.Longitude)]
fall_df['tree_id']=fall_tree_ids_list

In [ ]:
# save fallen tree ids so I don't need to run that function again
fall_df.to_csv('Data/Intermediate/tree_falls_with_id.csv', index=False)

In [ ]:
# export id columns too, just in case
tid_fall_df = pd.DataFrame(fall_df, columns=['tree_id'])
tid_fall_df.to_csv('Data/Intermediate/fall_tree_id.csv', index=False)

### Next, import 311 tree damage and overhanging tree service requests and assign each a street tree census tree_id
Follow the same process as above with tree falls.  Again only use data from before 2015.  Again remove all requests for trees in parks (not street trees).

In [31]:
# Import and organize 311 tree damage and overhanging branch service requests and drop unneeded columns
service_request_df = pd.read_csv('Input_Data/Tree_Service_Requests_311.csv',quoting=3)
service_request_df.drop([
       'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Status', 'Borough',
       'X Coordinate (State Plane)', 'Y Coordinate (State Plane)'],axis=1,inplace=True)

# Remove all service requests before 2015 (before the street tree census)
service_request_df['Created Date'] = pd.to_datetime(service_request_df['Created Date'])
service_request_df = service_request_df.where(service_request_df['Created Date'] > '2015-01-01')

# Remove all falls that occured in parks, and not in street trees
service_request_df = service_request_df.where(service_request_df['Location Type'] != 'Park')

Match each 311 branch fall service request with a street tree census tree_id using the assign_tree_id function defined above.  This matches trees based on address and lat-lon distance.

In [ ]:
# In order to match service requests to street tree census tree_ids, remove service requests without addresses or coordinates
service_request_df = service_request_df[service_request_df.Incident Address.isna()==False]
service_request_df = service_request_df[service_request_df.Latitude.isna()==False]
service_request_df = service_request_df[service_request_df.Longitude.isna()==False]

In [33]:
# Find closest tree from the street tree census and then assign its tree_index to the 311 data frame
tree_ids_service_requests = [assign_tree_id(address,lat,lon) for address,lat,lon in zip(service_request_df.Incident Address,service_request_df.Latitude,service_request_df.Longitude)]
service_request_df.tree_id=tree_ids_service_requests

In [51]:
# Save file with tree ids for damages
service_request_df.to_csv('Data/Intermediate/tree_service_requests_with_id.csv', index=False)

In [37]:
# export id columns too, just in case
tid_service_request_df = pd.DataFrame(service_request_df, columns=["tree_id"])
tid_service_request_df.to_csv('Data/Intermediate/service_request_tree_id.csv', index=False)

### Finally merge the street tree census data to branch fall data

In [39]:
# Join the tree fall df to the street tree census df

# First, drop rows without a tree_id from the branch fall df
fall_df = fall_df[fall_df.tree_id.isna()==False]

# now merge the branch fall df to the street tree census df
census_with_falls_df = tree_census_df.merge(fall_df,how="left",left_on="tree_id",right_on="tree_id")

In [47]:
# Finally, save this as a csv so we don't need to do it again
census_with_falls_df.to_csv('Data/Intermediate/tree_data_with_falls.csv', index=False)